In [158]:
import numpy as np
import pandas as pd
from scipy import fftpack
from scipy import signal
import datetime
%matplotlib inline

In [159]:
cd ~/Documents/SyncRecord/cleaning-addingLABEL/

/home/takeyama/Documents/SyncRecord/cleaning-addingLABEL


In [440]:
class RecordData:
    
    def __init__(self):
        print "__class__"
        self._fft_AccX=np.array([])
        self._fft_AccY=np.array([])
        self._fft_AccZ=np.array([])
        self._fft_GyrX=np.array([])
        self._fft_GyrY=np.array([])
        self._fft_GyrZ=np.array([])
        self._AccX=np.array([])
        self._AccY=np.array([])
        self._AccZ=np.array([])
        self._GyrX=np.array([])
        self._GyrY=np.array([])
        self._GyrZ=np.array([])
        self._Columns=np.array([])
        
    def ImportCSV(self,csv_file):
        # design dataframe
        self._data = pd.read_csv(csv_file,encoding="SHIFT-JIS")
        self._data.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
        self._data.Time=pd.to_datetime(self._data.Time)
        self._data = pd.pivot_table(self._data,values=[u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ'],index=[u'Time'])
        
        # convert numpy.darray
        self._AccX=data.AccX.values
        self._AccY=data.AccY.values
        self._AccZ=data.AccZ.values
        self._GyrX=data.GyrX.values
        self._GyrY=data.GyrY.values
        self._GyrZ=data.GyrZ.values
        self._Time=data.index.to_pydatetime().astype('datetime64[ns]')
        self._Columns=['Time','AccX','AccY','AccZ','GyrX','GyrY','GyrZ']
        self._fft_col=['fft_AccX','fft_AccY','fft_AccZ','fft_GyrX','fft_GyrY','fft_GyrZ']
        
    def GetColumns(self):
        return self._Columns

    def ShowAllDf(self):
        print self._AccX;print self._AccY;print self._AccZ;print self._GyrX;print self._GyrY;print self._GyrZ;print self._Time
        
    def _String2Var(self,Sname):
        if(Sname=='AccX'):return self._AccX
        elif(Sname=='AccY'):return self._AccY
        elif(Sname=='AccZ'):return self._AccZ
        elif(Sname=='GyrX'):return self._GyrX
        elif(Sname=='GyrY'):return self._GyrY
        elif(Sname=='GyrZ'):return self._GyrZ
        elif(Sname=='Time'):return self._Time
        elif(Sname=='fft_AccX'):return self._fft_AccX
        elif(Sname=='fft_AccY'):return self._fft_AccY
        elif(Sname=='fft_AccZ'):return self._fft_AccZ
        elif(Sname=='fft_GyrX'):return self._fft_GyrX
        elif(Sname=='fft_GyrY'):return self._fft_GyrY
        elif(Sname=='fft_GyrZ'):return self._fft_GyrZ
        else:print 'Wrong Axis Name'
    
    def _Time2Num(self,time):
        index=np.where(test._Time==np.datetime64(time) )
        return index[0][0]    
    
    def _Query(self,Sname,rng=[]):
        data = self._String2Var(Sname)
        return data[rng[0]:rng[1]]
            
    def GetLength(self,Sname):
        data = _String2Var(Sname)
        print Sname+' length ='+str( len(data) )
            
    def ShowQuery(self,Sname,rng=[]):
        data = self._String2Var(Sname)
        print Sname+':'+str( data[rng[0]:rng[1]])
            
    def _sliding_window(self,Sname,samp, overlap):
        count =0
        s =self._Time[0]
        start=self._Time2Num(s)
        g = s+np.timedelta64(samp*10,'ms')
        goal= self._Time2Num(g)
        yield self._Query(Sname,[start,goal])
        
        add=overlap*0.01
        
        while True:
            try:
                count +=1
                s =s+np.timedelta64(samp*10,'ms')
                start=self._Time2Num(s)
                g  =s+np.timedelta64(samp*10,'ms')
                goal= self._Time2Num(g)
                yield self._Query(Sname,[start,goal])
            except StopIteration:
                print '_sliding_window StopIteration'
                break   
            except IndexError:
                print '_sliding_window IndexError'
                break   
    def _set_fft(self,Sname,array):
        #print array
        if(Sname=='fft_AccX'):   self._fft_AccX=np.append(self._fft_AccX,np.array([array]) )
        elif(Sname=='fft_AccY'): self._fft_AccY=np.append(self._fft_AccY,np.array([array]) )
        elif(Sname=='fft_AccZ'): self._fft_AccZ=np.append(self._fft_AccZ,np.array([array]) )
        elif(Sname=='fft_GyrX'): self._fft_GyrX=np.append(self._fft_GyrX,np.array([array]) )
        elif(Sname=='fft_GyrY'): self._fft_GyrY=np.append(self._fft_GyrY,np.array([array]) )
        elif(Sname=='fft_GyrZ'): self._fft_GyrZ=np.append(self._fft_GyrZ,np.array([array]) )
        else:print 'Wrong Axis Name'
            
    def CalcFFT(self,samp,overlap=0.5):
        for n,f in zip( self._Columns[1:],self._fft_col):
            print 'start'+n+'->'+f
            sw = self._sliding_window(n,samp,overlap)
            count=0
            while True:
                try:
                    #print 'count ='+str(count)
                    d=sw.next()
                    # fftの直流成分を除くsample/2のみ
                    self._set_fft(f,fftpack.fft(d)[1:(samp/2)+1])     
                    count = count+1
                except StopIteration:
                    print 'CalcFFTStopIteration'
                    break   
        self._fft_AccX=self._fft_AccX.reshape(len(self._fft_AccX)/(samp/2),(samp/2) )
        self._fft_AccY=self._fft_AccY.reshape(len(self._fft_AccY)/(samp/2),(samp/2) )
        self._fft_AccZ=self._fft_AccZ.reshape(len(self._fft_AccZ)/(samp/2),(samp/2) )
        self._fft_GyrX=self._fft_GyrX.reshape(len(self._fft_GyrX)/(samp/2),(samp/2) )
        self._fft_GyrY=self._fft_GyrY.reshape(len(self._fft_GyrY)/(samp/2),(samp/2) )
        self._fft_GyrZ=self._fft_GyrZ.reshape(len(self._fft_GyrZ)/(samp/2),(samp/2) )
    #def CalcFFT(self,samp,overlap=0.5):
            
                
    def ShowFFT(self,name):
        data = self._String2Var(name)
        print data

SyntaxError: invalid syntax (<ipython-input-440-44fff64d0ad8>, line 124)

In [441]:
test = RecordData()
test.ImportCSV('west.csv')
test.CalcFFT(16)
#d=test._Query('AccX',[test._Time[0],test._Time[0]+datetime.timedelta(seconds=0.16)])
#fftpack.fft(test._AccX)
#print test._Time[0]
#print type(test._Time[0])
#print test._Time[0]+np.timedelta64(160,'ms')
#np.where(test._Time==np.datetime64('2016-05-20T18:06:15.908000000') )
#min(test._Time)> max(test._Time)
test.ShowFFT('fft_AccX')
test.ShowFFT('fft_AccY')
test.ShowFFT('fft_AccZ')
test.ShowFFT('fft_GyrX')
test.ShowFFT('fft_GyrY')
test.ShowFFT('fft_GyrZ')
test._fft_AccX.shape

__class__
startAccX->fft_AccX
_sliding_window IndexError
CalcFFTStopIteration
startAccY->fft_AccY
_sliding_window IndexError
CalcFFTStopIteration
startAccZ->fft_AccZ
_sliding_window IndexError
CalcFFTStopIteration
startGyrX->fft_GyrX
_sliding_window IndexError
CalcFFTStopIteration
startGyrY->fft_GyrY
_sliding_window IndexError
CalcFFTStopIteration
startGyrZ->fft_GyrZ
_sliding_window IndexError
CalcFFTStopIteration
[[ 0.32560971-0.16837066j  0.06307975-0.10497946j  0.05093345-0.05475064j
  ...,  0.02532025-0.00037946j  0.02975063-0.00937987j  0.05600000+0.j        ]
 [-0.23214476-0.19113553j -0.02342584+0.03783417j -0.00507246+0.00053545j
  ..., -0.00037416-0.00516583j  0.00105448-0.00114718j  0.01810000+0.j        ]
 [ 0.12049876-0.11881285j  0.06768305+0.00859899j  0.03543651-0.00484711j
  ...,  0.01931695+0.03099899j -0.01758362-0.10443194j -0.03470000+0.j        ]
 ..., 
 [-0.06101110-0.06334839j  0.08257666-0.02822864j -0.03634715-0.00618046j
  ...,  0.00762334-0.02522864j  0.00169

(349, 8)

In [216]:
data = pd.read_csv("Conv-west-ags.csv",encoding="Shift-JIS")
data.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
data.AccX = data.AccX*0.1*0.001
data.AccY = data.AccY*0.1*0.001
data.AccZ = data.AccZ*0.1*0.001

data.GyrX = data.GyrX*0.1*0.001
data.GyrY = data.GyrY*0.1*0.001
data.GyrZ = data.GyrZ*0.1*0.001

data.Time=pd.to_datetime(data.Time)

data = pd.pivot_table(data,values=[u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ'],index=[u'Time'])
AccX=data.AccX

In [431]:
2784/8

348